In [1]:
# Import libraries
import betfairlightweight
from betfairlightweight import filters
import pandas as pd
import numpy as np
import os
import datetime
import json
from utils import process_runner_books
import pytz
import time
from IPython.display import display

# Project libraries
import constants

# Change this certs path to wherever you're storing your certificates
certs_path = "/home/naing/certs/"
# Change these login details to your own
my_username = constants.USERNAME
my_password = constants.PASSWORD
my_app_key = constants.API_KEY_DEMO #demo
# my_app_key = constants.API_KEY_LIVE #live

trading = betfairlightweight.APIClient(username=my_username,
                                       password=my_password,
                                       app_key=my_app_key,
                                       certs=certs_path)

trading.login()

## TODO
debug = True # Print the data frames
completion_flag = False # completion flag

<LoginResource>

In [2]:

# liability_amount = 100

In [3]:
#######################################
# Filter out only greyhoud races
#######################################
gh_racing_id = constants.GH_RACING_ID #Greyhound racing ID
greyhound_racing_filter = filters.market_filter(
    event_type_ids=[gh_racing_id],
    market_countries=['GB'],
    market_type_codes=['FORECAST'],
    market_start_time={
        'to': (datetime.datetime.utcnow() + datetime.timedelta(hours=6)).strftime("%Y-%m-%dT%TZ") #sydtime 5pm to 7am next day for GB gh races
    }
)
#  This returns a list
gb_gh_events = trading.betting.list_events(
    filter=greyhound_racing_filter)

if debug:
    # Create a DataFrame with all the events by iterating over each event object
    gb_gh_events_df = pd.DataFrame({
        'Event Name': [event_object.event.name for event_object in gb_gh_events],
        'Event ID': [event_object.event.id for event_object in gb_gh_events],
        'Event Venue': [event_object.event.venue for event_object in gb_gh_events],
        'Country Code': [event_object.event.country_code for event_object in gb_gh_events],
        'Time Zone': [event_object.event.time_zone for event_object in gb_gh_events],
        'Open Date': [event_object.event.open_date for event_object in gb_gh_events],
        'Market Count': [event_object.market_count for event_object in gb_gh_events]
    })

    display(gb_gh_events_df)

,Event Name,Event ID,Event Venue,Country Code,Time Zone,Open Date,Market Count
0,Doncaster (F/C) 3rd Jul,31567440,Doncaster,GB,Europe/London,2022-07-03 09:46:00,1
1,Harlow (F/C) 3rd Jul,31567452,Harlow,GB,Europe/London,2022-07-03 09:53:00,1


In [4]:
#######################################
#TODO: extract a list of IDs for the forecast markets
#######################################
fc_venue_ids = []
fc_venue_names = []
for eventObj in gb_gh_events:
    fc_venue_ids.append(eventObj.event.id)
    fc_venue_names.append(eventObj.event.name)

fc_venue_ids, fc_venue_names


(['31567440', '31567452'], ['Doncaster (F/C) 3rd Jul', 'Harlow (F/C) 3rd Jul'])

In [5]:
#######################################
## Filter out the next 5 upcoming games
#######################################
market_catalogue_filter = filters.market_filter(
    event_ids=fc_venue_ids)

market_catalogues = trading.betting.list_market_catalogue(
    filter=market_catalogue_filter,
    market_projection= ['MARKET_START_TIME', 'EVENT'],
    max_results='5',
    sort='FIRST_TO_START'
)

## show the dataframes if set
if debug:
    market_types_venueOfTheDay_df = pd.DataFrame({
        'Market Name': [market_cat_object.market_name for market_cat_object in market_catalogues],
        'Market ID': [market_cat_object.market_id for market_cat_object in market_catalogues],
        'Market Start Time': [market_cat_object.market_start_time for market_cat_object in market_catalogues],
        'Total Matched': [market_cat_object.total_matched for market_cat_object in market_catalogues],
        'Venue': [market_cat_object.event.venue for market_cat_object in market_catalogues]
        
    })

    display(market_types_venueOfTheDay_df)

,Market Name,Market ID,Market Start Time,Total Matched,Venue
0,Forecast,1.200715847,2022-07-03 09:46:00,0.0,Doncaster
1,Forecast,1.200715883,2022-07-03 09:53:00,0.0,Harlow
2,Forecast,1.200715848,2022-07-03 10:01:00,0.0,Doncaster
3,Forecast,1.200715884,2022-07-03 10:09:00,0.0,Harlow
4,Forecast,1.200715849,2022-07-03 10:16:00,0.0,Doncaster


In [6]:
#######################################
## Check the start time of the next game
## and sleep until 15 seconds before the game
#######################################
from utils import get_next_market

time_gap, myRaceID, myRaceVenue = get_next_market(market_catalogues)
# assert myRaceID == 0, "ERROR: myRaceID = 0" #TODO: remove in production

if(time_gap > datetime.timedelta(seconds=constants.PREBET_DELAY)):
    time_to_sleep = (time_gap - datetime.timedelta(seconds=constants.PREBET_DELAY)).seconds
    print("Sleeping for " + str(time_to_sleep) + " seconds")
    # time.sleep(time_to_sleep)
    print("Sleeping done")
else:
    print("Don't need to sleep")



Time Now: 
2022-07-03 03:54:47
Found the market to lay: Name = doncaster id = 1.200715847
Market Start Time: 2022-07-03 09:46:00+00:00
Sleeping for 21052 seconds
Sleeping done


In [7]:
#######################################
# Get a list of lay options
#######################################
# Create a price filter. Get all traded and offer data
price_filter = filters.price_projection(
    price_data=['EX_BEST_OFFERS']
)

# Request market books
market_books = trading.betting.list_market_book(
    market_ids=[myRaceID],
    price_projection=price_filter
)

# Grab the first market book from the returned list as we only requested one market 
market_book = market_books[0]
runners_df = process_runner_books(market_book.runners)
if debug:
    display(runners_df)

#TODO: extract a list of IDs for the lay options
lay_options_ids = []
for obj in market_book.runners:
    lay_options_ids.append(obj.selection_id)
    
# if debug:
#     print(lay_options_ids)

#######################################
# Get odds from NEDS
#######################################
from utils import choose_lay_option

lay_selection_index = choose_lay_option(myRaceVenue)

if(lay_selection_index == -1):
    print("ERROR!!! 2 same odds found")
    assert()
    
elif(lay_selection_index == -2):
    print("ERROR!!! less than 6 runners")
    # assert()

elif(lay_selection_index == -3):
    print("ERROR!!! NEDS API failed")
    assert()
else:
    print("Lay option found successfully")
    print(lay_selection_index)

lay_selection_id = lay_options_ids[lay_selection_index]
print("lay_selection_id = " + str(lay_selection_id))
fav_price = runners_df.loc[runners_df['Best Lay Price'].idxmin(), 'Best Lay Price']



,Selection ID,Best Back Price,Best Back Size,Best Lay Price,Best Lay Size,Last Price Traded,Total Matched,Status,Removal Date,Adjustment Factor
0,41419585,1.01,1.01,1000.0,1.01,None,None,ACTIVE,None,None
1,41419586,1.01,1.01,1000.0,1.01,None,None,ACTIVE,None,None
2,41419587,1.01,1.01,1000.0,1.01,None,None,ACTIVE,None,None
3,41419588,1.01,1.01,1000.0,1.01,None,None,ACTIVE,None,None
4,41419589,1.01,1.01,1000.0,1.01,None,None,ACTIVE,None,None
5,41419592,1.01,1.01,1000.0,1.01,None,None,ACTIVE,None,None
6,41419593,1.01,1.01,1000.0,1.01,None,None,ACTIVE,None,None
7,41419594,1.01,1.01,1000.0,1.01,None,None,ACTIVE,None,None
8,41419595,1.01,1.01,1000.0,1.01,None,None,ACTIVE,None,None
9,41419596,1.01,1.01,1000.0,1.01,None,None,ACTIVE,None,None


['DONCASTER BAGS', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '483m', '483Mtrs (A2)', '5h', 'RACE FIELD', 'COMMENTS', '$ TRACKER', 'WIN / PLACE', 'LIVE VIDEO', 'Doncaster Bags', 'RUNNERS']
end of part1
['RUNNERS', 'SP', '2624', '1. Fill The Till (1)', 'T: J W Gaskin', 'SP', '6414', '2. Burnfoot Roxy (2)', 'T: C J Lister', 'SP', '442', '3. Church St Millie (3)', 'T: S Watson', 'SP', '2416', '4. Kinte Kunta (4)', 'T: S Watson', 'SP', '3223', '5. Pennys Endeavour (5)', 'T: L J Macmanus', 'SP', '256', '6. Millcroft Moveit (6)', 'T: G Conway', 'SP', 'Event Rules: Only two place dividends paid.', 'BUSAN KOREA  R2', 'CLASS 6 SPECIAL WEIGHT A STAKES', '1199M', 'Jumps in', '8s', 'NEXT TO GO']
True Venue Name = doncaster-bags
['SP', 'SP', 'SP', 'SP', 'SP', 'SP']


ValueError: could not convert string to float: 'SP'

In [3]:
#######################################
# Choose the liability amount
# random from a list of 10 options
#######################################
import numpy as np
import constants
liability_options = []

if(len(liability_options) == 0):
    liability_options = [20, 20, 20, 20, 20, 20, 20, 20, 100, 100] #not sure why this can't be moved into constants.py
print("liability_options [BEFORE] = ", liability_options, ", LENGTH =", len(liability_options))

[liability_amount] = np.random.choice(liability_options, size=1)
print("Chosen Liability amount = $", liability_amount)

liability_options.remove(liability_amount)
print("liability_options [AFTER] = ", liability_options, ", LENGTH =", len(liability_options))

liability_options [BEFORE] =  [20, 20, 20, 20, 20, 20, 20, 20, 100, 100] , LENGTH = 10
Chosen Liability amount = $ 20
liability_options [AFTER] =  [20, 20, 20, 20, 20, 20, 20, 100, 100] , LENGTH = 9


In [ ]:
#######################################
# Choose order type (limit order or market_on_close order - choose the latter)
#######################################
order_filter = filters.limit_order(
    price=fav_price,
    persistence_type='LAPSE',
    bet_target_type='PAYOUT',
    bet_target_size=liability_amount 
)
print(order_filter)

#######################################
# Create a place instruction filter
#######################################
instructions_filter = filters.place_instruction(
    selection_id = str(lay_selection_id),
    side="LAY",
    order_type = "LIMIT", # fixed price order
    limit_order=order_filter
)
print(instructions_filter)


In [45]:
#######################################
# Place the order
#######################################
order = trading.betting.place_orders(
    market_id = myRaceID,
    customer_strategy_ref='Naing_maker',
    instructions=[instructions_filter]
)

In [2]:
#######################################
# List the cleared orders
#######################################
import utils
myRaceID = 1.200450524 # lost raceID

# time.sleep(5)
# current_orders = trading.betting.list_current_orders(
#     customer_strategy_refs=['Naing_maker']).__dict__
# data = pd.DataFrame(current_orders['currentOrders']).head()
# print(current_orders)

#######################################
# Check if the last bet has settled and note the result
#######################################
settled_flag = False
while settled_flag == False:
    cleared_orders = trading.betting.list_cleared_orders(
        bet_status="SETTLED",
        market_ids=[myRaceID])

    if len(cleared_orders._data['clearedOrders']) != 0:
        settled_flag = True

        # Create a DataFrame from the orders
        betResult = pd.DataFrame(cleared_orders._data['clearedOrders'])
        print(betResult)

        betOutcome = betResult['betOutcome'][0]
        profit = 0
        for i in betResult['profit']:
            profit = profit + float(i)

        ## Record the results into a csv file
        utils.write_to_file(constants.F_NAME, myRaceID, betOutcome, profit)

        completion_flag = True # we are ready for next game

    else:
        print("Sleep 60 seconds before checking again if market is settled")
        time.sleep(5) # TODO:Check again in 60 seconds
        
    



  eventTypeId   eventId     marketId  selectionId  handicap         betId  \
0        4339  31548500  1.200450524     41419593       0.0  272264823796   
1        4339  31548500  1.200450524     41419593       0.0  272264846627   
2        4339  31548500  1.200450524     41419593       0.0  272264865980   
3        4339  31548500  1.200450524     41419593       0.0  272264895029   

                 placedDate persistenceType orderType side betOutcome  \
0  2022-06-24T13:36:39.000Z           LAPSE     LIMIT  LAY       LOST   
1  2022-06-24T13:36:56.000Z           LAPSE     LIMIT  LAY       LOST   
2  2022-06-24T13:37:09.000Z           LAPSE     LIMIT  LAY       LOST   
3  2022-06-24T13:37:24.000Z           LAPSE     LIMIT  LAY       LOST   

   priceRequested               settledDate           lastMatchedDate  \
0            16.0  2022-06-24T13:42:43.000Z  2022-06-24T13:36:39.000Z   
1            18.0  2022-06-24T13:42:43.000Z  2022-06-24T13:36:58.000Z   
2            18.0  2022-06-24